In [21]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print("当前设备名称：", torch.cuda.get_device_name(device) if device.type == 'cuda' else 'CPU')

In [22]:
def image2emb_naive(image, patch_size, weight):
    patch = F.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1, -2)
    print(patch.shape)
    patch_embedding = patch @ weight
    return patch_embedding
# 通过卷积实现
def image2emb_conv(image, kernel, stride):
    # kernel训练中更新参数 
    # oc*ic*kh*kw
    conv_output = F.conv2d(image, kernel, stride=stride)
    ba, oc, oh, ow = conv_output.shape
    patch_embedding = conv_output.reshape((ba, oc, oh*ow)).transpose(-1,-2)
    return patch_embedding
# test code for image2emb
bs, ic, image_h, image_w = 1, 3, 8, 8
patch_size = 4
model_dim = 8
patch_depth = patch_size*patch_size*ic
image = torch.randn(bs, ic, image_h, image_w)
weight = torch.randn(patch_depth, model_dim)
patch_embedding_naive = image2emb_naive(image, patch_size, weight)
print(patch_embedding_naive.size())
print(patch_embedding_naive)
##################################
kernel = weight.transpose(0, 1).reshape((model_dim, ic, patch_size, patch_size))
patch_embedding_conv = image2emb_conv(image, kernel, patch_size)
print(patch_embedding_conv.size())
print(patch_embedding_conv)
torch.allclose(patch_embedding_naive, patch_embedding_conv)

torch.Size([1, 4, 48])
torch.Size([1, 4, 8])
tensor([[[  7.4667,  -5.5059,   9.3193,  -0.0413,  -9.4175,   6.5571,   4.1736,
            5.8958],
         [ -2.7264,  -5.8506,   9.3051,   0.1965,   5.9004,   2.5408,  -2.7052,
          -14.4034],
         [-11.3469,   0.3776,  -4.8890,  -6.0610,  -3.0947,   8.6485,  -5.2425,
            3.1319],
         [ 15.7369,  -9.9057,  -6.3652,   7.2953,  -3.1388,   0.9963,  -2.1650,
           -4.0748]]])
torch.Size([1, 4, 8])
tensor([[[  7.4667,  -5.5059,   9.3193,  -0.0413,  -9.4175,   6.5571,   4.1736,
            5.8958],
         [ -2.7264,  -5.8506,   9.3051,   0.1965,   5.9004,   2.5408,  -2.7052,
          -14.4034],
         [-11.3469,   0.3776,  -4.8890,  -6.0610,  -3.0947,   8.6485,  -5.2425,
            3.1319],
         [ 15.7369,  -9.9057,  -6.3652,   7.2953,  -3.1388,   0.9963,  -2.1650,
           -4.0748]]])
